In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompt Design - Best Practices

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fprompts%2Fintro_prompt_design.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | | |
|-|-|-|
|Author(s) | [Polong Lin](https://github.com/polong-lin) | [Karl Weinmeister](https://github.com/kweinmeister)|

## Overview

This notebook covers the essentials of prompt engineering, including some best practices.

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

In this notebook, you learn best practices around prompt engineering -- how to design prompts to improve the quality of your responses. 

This notebook covers the following best practices for prompt engineering:

- Be concise
- Be specific and well-defined
- Ask one task at a time
- Turn generative tasks into classification tasks
- Improve response quality by including examples

## Getting Started

### Install Vertex AI SDK and other required packages


In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [1]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "qwiklabs-gcp-02-a93657cd2b30"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [3]:
from vertexai.generative_models import GenerationConfig, GenerativeModel
import time

### Load model

In [4]:
model = GenerativeModel("gemini-1.5-flash")

In [5]:
import time

def call_gemini(prompt, generation_config=GenerationConfig(temperature=1.0)):
    wait_time = 1
    while True:
        try:
            response = model.generate_content(prompt, generation_config=generation_config).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

def send_message_gemini(model, prompt):    
    wait_time = 1
    while True:
        try:
            response = model.send_message(prompt).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

## Prompt engineering best practices

Prompt engineering is all about how to design your prompts so that the response is what you were indeed hoping to see.

The idea of using "unfancy" prompts is to minimize the noise in your prompt to reduce the possibility of the LLM misinterpreting the intent of the prompt. Below are a few guidelines on how to engineer "unfancy" prompts.

In this section, you'll cover the following best practices when engineering prompts:

* Be concise
* Be specific, and well-defined
* Ask one task at a time
* Improve response quality by including examples
* Turn generative tasks to classification tasks to improve safety

### Be concise

🛑 Not recommended. The prompt below is unnecessarily verbose.

In [6]:
prompt = "What do you think could be a good name for a flower shop that specializes in selling bouquets of dried flowers more than fresh flowers?"

print(call_gemini(prompt))

Here are some names for a flower shop specializing in dried flowers, playing with different themes and styles:

**Whimsical & Romantic:**

* Whispers of Bloom
* The Dried Petal
* Evergreen & Lace
* Bloom & Bloom
* Petal & Stone
* The Wildflower Co.
* Bloom & Co.
* The Dried Bouquet
* The Flower Whisperer

**Modern & Minimalist:**

* Flora & Fauna
* The Bloom Bar
* Bloom & Branch
* The Paper Petal
* The Dried Garden
* The Flower Lab
* Botanical & Co.
* The Dried Bloom

**Rustic & Earthy:**

* The Rustic Bouquet
* The Dried Field
* Gather & Bloom
* The Wildflower Barn
* The Dried Meadow
* The Willow Branch
* The Country Bloom

**Unique & Playful:**

* The Petal Press
* The Flower Alchemist
* The Flower Archivist
* The Bloom & Bloom
* The Botanical Cabinet
* The Flower Vault

**Tips for Choosing a Name:**

* **Consider your target audience:** Who are you trying to attract?
* **Keep it memorable and easy to pronounce:** You want people to remember your name.
* **Make sure the name is avail

✅ Recommended. The prompt below is to the point and concise.

In [7]:
prompt = "Suggest a name for a flower shop that sells bouquets of dried flowers"

print(call_gemini(prompt))

##  Flower Shop Names for Dried Bouquets:

**Evocative & Romantic:**

* Whispers of Bloom
* Forever in Bloom
* The Dried Bloom
* Everlasting Blooms
* Bloom & Beyond
* Dried & Delightful
* Petal & Paper
* Timeless Blooms
* The Dried Bouquet
* Bloom & Co.

**Unique & Playful:**

* The Flower Muse
* Poppy & Thistle
* Petal Preservation
* Bloom & Preserve
* The Dried Garden
* Wildflower & Co.
* Botanical Reverie
* Petal Past
* The Dried Petal
* The Bloom Box

**Modern & Minimalist:**

* Dried Florals
* Bloom Lab
* Bloom Co.
* The Flower Atelier
* Botanical Collective
* Bloom & Co.
* The Dried Bouquet Co.
* Bloom & Co.
* The Flower Studio
* Bloom & More

**Location-Based:**

* [Your City] Dried Florals
* [Your City] Bloom & Co.
* The [Your Neighborhood] Flower Shop
* [Your Street Name] Dried Flowers

**Tips for Choosing a Name:**

* **Keep it short and memorable.**
* **Make it relevant to your products.**
* **Consider your target audience.**
* **Check if the name is available as a domain na

### Be specific, and well-defined

Suppose that you want to brainstorm creative ways to describe Earth.

🛑 The prompt below might be a bit too generic (which is certainly OK if you'd like to ask a generic question!)

In [8]:
prompt = "Tell me about Earth"

print(call_gemini(prompt))

## Earth: Our Home Planet

Earth is a vibrant and complex planet, the only known place in the universe that we know harbors life. Here are some key facts about our home:

**Physical Characteristics:**

* **Third planet from the Sun:** Orbiting at a distance of 93 million miles, Earth receives just the right amount of energy to support life.
* **Rocky planet:** Composed primarily of iron, nickel, silicon, and oxygen.
* **Atmosphere:** A protective blanket of gases, primarily nitrogen and oxygen, that shields us from harmful radiation and regulates temperature.
* **Water:** Covering 71% of Earth's surface, water is essential for life and plays a crucial role in weather patterns and climate.
* **Plate tectonics:** The Earth's crust is divided into massive plates that constantly move, creating mountains, volcanoes, and earthquakes.
* **Magnetic field:** Generated by the Earth's core, the magnetic field protects us from the Sun's harmful solar wind.

**Life on Earth:**

* **Biosphere:** The

✅ Recommended. The prompt below is specific and well-defined.

In [9]:
prompt = "Generate a list of ways that makes Earth unique compared to other planets"

print(call_gemini(prompt))

## Earth: A Unique Jewel in the Solar System

Here are some aspects that make Earth stand out among other planets:

**1. Liquid Water:** Earth is the only known planet with **abundant liquid water** on its surface. This is crucial for life as we know it, playing a vital role in chemical reactions, transportation, and temperature regulation.

**2. Plate Tectonics:**  Earth's **dynamic tectonic plates** constantly shift and interact, causing earthquakes, volcanoes, and mountain formation. This geological activity recycles the planet's crust and releases heat from the interior, creating a diverse and ever-changing environment.

**3. Atmosphere and Climate:** Earth possesses a **stable and breathable atmosphere**, composed mainly of nitrogen and oxygen. This atmosphere protects us from harmful solar radiation, creates weather patterns, and moderates temperatures.

**4. Biosphere:**  Earth is the only known planet to harbor a **thriving biosphere** with diverse ecosystems, a wide variety of

### Ask one task at a time

🛑 Not recommended. The prompt below has two parts to the question that could be asked separately.

In [10]:
prompt = "What's the best method of boiling water and why is the sky blue?"

print(call_gemini(prompt))

Let's tackle these two very different questions!

**Boiling Water**

The best method of boiling water depends on what you're using it for and the resources you have. Here's a breakdown of common methods:

* **Stovetop:** This is the most common method for everyday use. Simply fill a pot with water, place it on a burner, and turn it on high heat.
* **Microwave:** This is quick and convenient for small amounts of water, but not ideal for larger volumes. 
* **Electric Kettle:** This is fast and energy-efficient, perfect for making tea or coffee.
* **Camping Stove:** For outdoor use, a camping stove with a pot or kettle works well.

**Why is the Sky Blue?**

The sky appears blue due to a phenomenon called **Rayleigh scattering**. Here's the explanation:

* **Sunlight is made up of all colors:**  White sunlight is actually a mix of all colors of the rainbow.
* **Tiny particles scatter light:**  When sunlight enters the Earth's atmosphere, it interacts with tiny particles like nitrogen and o

✅ Recommended. The prompts below asks one task a time.

In [11]:
prompt = "What's the best method of boiling water?"

print(call_gemini(prompt))

There's no single "best" method, as the ideal approach depends on what you're using the boiling water for and what resources you have. Here are some common methods, each with its pros and cons:

**1. Stovetop:**

* **Pros:** Most common and accessible, good for larger quantities.
* **Cons:** Can be slow, requires constant monitoring, potential for spills.

**2. Electric Kettle:**

* **Pros:** Fastest and most efficient, convenient for smaller amounts.
* **Cons:** Can be bulky, not ideal for large quantities.

**3. Microwave:**

* **Pros:** Very fast for small amounts, convenient for single servings.
* **Cons:** Not as efficient for larger amounts, can be uneven heating.

**4. Camping Stove:**

* **Pros:** Portable and versatile, great for outdoor use.
* **Cons:** Requires fuel, can be less efficient than other methods.

**Here's a breakdown of considerations:**

* **Quantity:** For larger quantities, a stovetop or camping stove is best. For small amounts, a kettle or microwave is more 

In [ ]:
prompt = "Why is the sky blue?"

print(call_gemini(prompt))

### Watch out for hallucinations

Although LLMs have been trained on a large amount of data, they can generate text containing statements not grounded in truth or reality; these responses from the LLM are often referred to as "hallucinations" due to their limited memorization capabilities. Note that simply prompting the LLM to provide a citation isn't a fix to this problem, as there are instances of LLMs providing false or inaccurate citations. Dealing with hallucinations is a fundamental challenge of LLMs and an ongoing research area, so it is important to be cognizant that LLMs may seem to give you confident, correct-sounding statements that are in fact incorrect.

Note that if you intend to use LLMs for the creative use cases, hallucinating could actually be quite useful.

Try the prompt like the one below repeatedly. We set the temperature to 1.0 so that it takes more risks in its choices. It's possible that it may provide an inaccurate, but confident answer.

In [13]:
generation_config = GenerationConfig(temperature=1.0)

prompt = "What day is it today?"

print(call_gemini(prompt, generation_config))

I do not have access to real-time information, including the current date. To know what day it is today, please check a calendar or your device's date settings. 



Since LLMs do not have access to real-time information without further integrations, you may have noticed it hallucinates what day it is today in some of the outputs.

### Using system instructions to guardrail the model from irrelevant responses

How can we attempt to reduce the chances of irrelevant responses and hallucinations?

One way is to provide the LLM with [system instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini#system-instructions).

Let's see how system instructions works and how you can use them to reduce hallucinations or irrelevant questions for a travel chatbot.

Suppose we ask a simple question about one of Italy's most famous tourist spots.

In [14]:
model_travel = GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=[
        "Hello! You are an AI chatbot for a travel web site.",
        "Your mission is to provide helpful queries for travelers.",
        "Remember that before you answer a question, you must check to see if it complies with your mission.",
        "If not, you can say, Sorry I can't answer that question.",
    ],
)

chat = model_travel.start_chat()

prompt = "What is the best place for sightseeing in Milan, Italy?"

print(send_message_gemini(chat, prompt))

Milan is a vibrant city with a rich history and culture.  There are many places to see, depending on your interests. Some popular options include:

* **Duomo di Milano:** The magnificent Duomo, a Gothic cathedral with stunning architecture and intricate details.
* **Galleria Vittorio Emanuele II:** A grand shopping arcade with beautiful glass ceilings and high-end stores.
* **Sforza Castle:** A medieval castle with museums, art galleries, and scenic gardens.
* **Santa Maria delle Grazie:** Home to Leonardo da Vinci's iconic painting, The Last Supper.
* **Teatro alla Scala:** A world-renowned opera house with impressive performances and a rich history.

Do you have any specific interests in mind? For example, are you looking for historical sites, art museums, shopping, or something else?  Knowing your preferences would help me provide more tailored recommendations. 



Now let us pretend to be a user asks the chatbot a question that is unrelated to travel.

In [15]:
prompt = "What's for dinner?"

print(send_message_gemini(chat, prompt))

Sorry, I can't answer that question. I'm an AI chatbot for a travel website, so I'm not able to provide restaurant recommendations or meal suggestions. 



You can see that this way, a guardrail in the prompt prevented the chatbot from veering off course.

### Turn generative tasks into classification tasks to reduce output variability

#### Generative tasks lead to higher output variability

The prompt below results in an open-ended response, useful for brainstorming, but response is highly variable.

In [16]:
prompt = "I'm a high school student. Recommend me a programming activity to improve my skills."

print(call_gemini(prompt))

Here are a few programming activity ideas tailored for high school students, focusing on different aspects of skill development:

**1. Build a Simple Game:**

* **Skill Focus:**  Problem-solving, logic, game design basics, event handling, user input.
* **Project Ideas:**
    * **Text-based adventure game:**  Use simple text prompts and user input to create a story-driven game.
    * **Simple arcade game:**  Create a classic game like Pong, Snake, or Tetris using graphics and game logic.
    * **Trivia game:**  Build a quiz with multiple-choice questions and scoring.
* **Tools:**  Consider using Python (Pygame library), JavaScript (with HTML5 Canvas), or Scratch for a visual interface.

**2. Create a Personal Website:**

* **Skill Focus:**  HTML, CSS, basic JavaScript, web design principles, project organization.
* **Project Ideas:**
    * **Portfolio website:**  Showcase your skills, projects, and interests.
    * **Informational website:**  Create a website about a topic you're passio

#### Classification tasks reduces output variability

The prompt below results in a choice and may be useful if you want the output to be easier to control.

In [17]:
prompt = """I'm a high school student. Which of these activities do you suggest and why:
a) learn Python
b) learn JavaScript
c) learn Fortran
"""

print(call_gemini(prompt))

It's great that you're thinking about learning programming! All three languages have their merits, but here's a breakdown to help you decide:

**a) Learn Python:**

* **Why it's great:**
    * **Beginner-friendly:** Python is known for its clear syntax, making it easier to learn than some other languages. 
    * **Versatile:** Python is widely used for web development, data science, machine learning, scripting, and more. 
    * **Strong community:** Python has a large and active community, meaning you'll find plenty of resources and support if you need it.
* **Consider this:** If you're interested in areas like data science, machine learning, or web development, Python is a great choice to start with.

**b) Learn JavaScript:**

* **Why it's great:**
    * **Interactive web development:** JavaScript is the language of the web, used to create interactive elements, animations, and dynamic websites.
    * **High demand:** JavaScript developers are in high demand, making it a valuable skill

### Improve response quality by including examples

Another way to improve response quality is to add examples in your prompt. The LLM learns in-context from the examples on how to respond. Typically, one to five examples (shots) are enough to improve the quality of responses. Including too many examples can cause the model to over-fit the data and reduce the quality of responses.

Similar to classical model training, the quality and distribution of the examples is very important. Pick examples that are representative of the scenarios that you need the model to learn, and keep the distribution of the examples (e.g. number of examples per class in the case of classification) aligned with your actual distribution.

#### Zero-shot prompt

Below is an example of zero-shot prompting, where you don't provide any examples to the LLM within the prompt itself.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment:
"""

print(call_gemini(prompt))

#### One-shot prompt

Below is an example of one-shot prompting, where you provide one example to the LLM within the prompt to give some guidance on what type of response you want.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment:
"""

print(call_gemini(prompt))

#### Few-shot prompt

Below is an example of few-shot prompting, where you provide a few examples to the LLM within the prompt to give some guidance on what type of response you want.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment: negative

Tweet: Something surprised me about this video - it was actually original. It was not the same old recycled stuff that I always see. Watch it - you will not regret it.
Sentiment:
"""

print(call_gemini(prompt))

#### Choosing between zero-shot, one-shot, few-shot prompting methods

Which prompt technique to use will solely depends on your goal. The zero-shot prompts are more open-ended and can give you creative answers, while one-shot and few-shot prompts teach the model how to behave so you can get more predictable answers that are consistent with the examples provided.